In [1]:
import pandas as pd

train_df = pd.read_csv("train_preprocessed_columns_dropped.csv")

train_df.head()

,tanggal_pencairan,jenis_pinjaman,status_peminjam,jumlah_pinjaman,total_pengembalian,durasi_hari,porsi_pengembalian_lender,status_gagal_bayar
0,8613,0.024529,0.042784,959000.0,959000.0,7.0,287000.0,0
1,8624,0.024529,0.042784,55000.0,55000.0,7.0,16000.0,0
2,8617,0.024529,0.042784,655000.0,666000.0,7.0,199000.0,0
3,9432,0.197461,0.042784,611000.0,632000.0,7.0,126000.0,0
4,8652,0.024529,0.042784,573000.0,573000.0,7.0,127000.0,0


In [2]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15214 entries, 0 to 15213
Data columns (total 8 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   tanggal_pencairan          15214 non-null  int64  
 1   jenis_pinjaman             15214 non-null  float64
 2   status_peminjam            15214 non-null  float64
 3   jumlah_pinjaman            15214 non-null  float64
 4   total_pengembalian         15214 non-null  float64
 5   durasi_hari                15214 non-null  float64
 6   porsi_pengembalian_lender  15214 non-null  float64
 7   status_gagal_bayar         15214 non-null  int64  
dtypes: float64(6), int64(2)
memory usage: 951.0 KB


In [3]:
from sklearn.utils import resample

# Separate majority and minority classes
df_majority = train_df[train_df.status_gagal_bayar == 0]
df_minority = train_df[train_df.status_gagal_bayar == 1]

# Upsample minority class
df_minority_upsampled = resample(df_minority, 
                               replace=True,     # sample with replacement
                               n_samples=len(df_majority),    # to match majority class
                               random_state=42) # reproducible results

# Combine majority class with upsampled minority class
train_df = pd.concat([df_majority, df_minority_upsampled])

# Display new class counts
print(train_df.status_gagal_bayar.value_counts())

x = train_df.drop(columns=['status_gagal_bayar'])
y = train_df['status_gagal_bayar']

status_gagal_bayar
0    14539
1    14539
Name: count, dtype: int64


In [5]:
from sklearn.model_selection import train_test_split

# Split for validation
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.25, random_state=42)

In [6]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.impute import SimpleImputer

In [ ]:
import xgboost as xgb

# Train model
xgb_model = xgb.XGBClassifier(random_state=42)
xgb_model.fit(x_train, y_train)

# Validation
y_pred = xgb_model.predict(x_val)
print(classification_report(y_val, y_pred))

[LightGBM] [Info] Number of positive: 10890, number of negative: 10918
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000782 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1037
[LightGBM] [Info] Number of data points in the train set: 21808, number of used features: 7
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.499358 -> initscore=-0.002568
[LightGBM] [Info] Start training from score -0.002568
              precision    recall  f1-score   support

           0       1.00      0.97      0.99      3621
           1       0.97      1.00      0.99      3649

    accuracy                           0.99      7270
   macro avg       0.99      0.99      0.99      7270
weighted avg       0.99      0.99      0.99      7270



In [20]:
df = pd.read_csv("test_preprocessed with impute mean with columns dropped.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5003 entries, 0 to 5002
Data columns (total 8 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id_transaksi               5003 non-null   int64  
 1   tanggal_pencairan          5003 non-null   int64  
 2   jenis_pinjaman             5003 non-null   float64
 3   status_peminjam            5003 non-null   float64
 4   jumlah_pinjaman            5003 non-null   float64
 5   total_pengembalian         5003 non-null   float64
 6   durasi_hari                5003 non-null   float64
 7   porsi_pengembalian_lender  5003 non-null   float64
dtypes: float64(6), int64(2)
memory usage: 312.8 KB


In [ ]:
df = pd.read_csv("test_preprocessed with impute mean with columns dropped.csv")

test_preds = xgb_model.predict(df.drop(columns=['id_transaksi']))

# Tambahkan ke dataframe
df['status_gagal_bayar'] = test_preds

# Drop every column except 'id_transaksi' and 'status_gagal_bayar'
df = df[['id_transaksi', 'status_gagal_bayar']]

In [22]:
df.head()

,id_transaksi,status_gagal_bayar
0,20013,0
1,20014,0
2,20015,0
3,20016,0
4,20017,0


In [23]:
df.to_csv("submission_lgbm_impute mean with columns dropped and oversampling.csv", index=False)